# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract text from a technical PDF
2. Prepare training data for causal language modelling (CLM)
3. Fine-tune a language (llama 3.2 3b) base model using QLoRA
4. Apply the QLoRA and the learned weights to the instruct model
5. Answer test questions

In [1]:
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

Cloning into 'mistralpeft'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 101 (delta 60), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 7.61 MiB | 38.60 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [2]:
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git pull 

/kaggle/working/mistralpeft
Already up to date.


In [3]:
!pip install -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.6 MB/s eta 0:00:00


In [4]:
from mistralpeft.utils import TextExtractor, prepare_for_training, generate_response, CLAPreprocessor 
from transformers import Trainer, TrainingArguments, AutoTokenizer
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
from peft import PeftConfig
from peft import PeftModel  # Make sure you've imported PeftModel

Switch of tracking with Weights And Biases (it requires an account)

In [5]:
os.environ["WANDB_DISABLED"] = "true"

Login to HuggingFace using Kaggle's secrets

In [6]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
login(secret_value_0) 

## 1. Extract Sentences from PDF

First, we'll extract and clean sentences from the dissertation PDF.

In [7]:
pdf_files = [
    "Dissertation.pdf",
    # "document2.pdf"
]
    
pdf_urls = [
    # "https://mediatum.ub.tum.de/doc/1360567/1360567.pdf",
    # "https://mediatum.ub.tum.de/doc/1601190/1601190.pdf",
    # "https://example.com/doc1.pdf",
    # "https://example.com/doc2.pdf"
]
    
with TextExtractor("output/processed_documents.json") as extractor:
    # Process local files
    extractor.process_documents(pdf_files)
        
    # Process URLs
    extractor.process_documents(pdf_urls, url_list=True)

Processing documents: 100%|██████████| 1/1 [00:09<00:00,  9.85s/it]
Processing documents: 0it [00:00, ?it/s]


## 2. Prepare MCLM Training Data

Now we'll create masked language modeling examples for training.

In [8]:
# Specify the model and load the tokenizer
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [9]:
json_file_paths = ["output/processed_documents.json"]
preprocessor = CLAPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Token indices sequence length is longer than the specified maximum sequence length for this model (152682 > 131072). Running this sequence through the model will result in indexing errors


In [10]:
# Split into training and test set
train_test_set = dataset.train_test_split(test_size=0.1)
print(f"Created {len(train_test_set)} training examples and {len(train_test_set)} test examples")

# Preview a training example
example = train_test_set["train"][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids'][:256]))

Created 2 training examples and 2 test examples

Example input:
 occur at the openings, and the tube serves as a half-wave resonator. The fundamental wavelength λ, neglecting end effects, is approximately twice the length of the tube λ = 2 L. The Strouhal number of the fundamental frequency is hence given by Sr = πd L Ma. (9.01) Higher harmonics are integer multiples of this frequency. Although higher harmon- ics may occur, the focus is given to the fundamental mode only. The temporal evolution of acoustic pressure and velocity within one cycle is shown in Figure 9.02. In the first graph, fluid exits the resonator at both ends. The velocity is positive with respect to the spatial coordinate direction x in the upper half of the resonator and negative in the lower half. The pressure in the tube decreases until the acoustic velocity reverses. This leads to a rise in pressure and in turn to the flow reversing again which closes the circle. This can be depicted in frequency space as spatial

## 3. Load and Prepare Model

Load the base model and prepare it for QLoRA fine-tuning.

Q4_K_M quantization of the base model is achieved through _BitsAndBytes_. It requires CUDA.

In [11]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors
    )

base_model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, device_map="auto",
    quantization_config=quantization_config,
    )

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Configure the (Q)LoRA adaptor to use a rank of $r=4$

In [12]:
model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

## 4. Train the Model

Now we'll fine-tune the model on our domain-specific data.

Training the LoRA adaptor on the thermo-fluiddynamics dataset using a paged AdamW 8 bit optimizer for memory efficiency.

In [13]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit",
    log_level="warning"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,3.354500,3.873968
2,2.781700,3.316235
3,3.073500,3.218453


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=102, training_loss=4.316118636552026, metrics={'train_runtime': 1910.3693, 'train_samples_per_second': 0.053, 'train_steps_per_second': 0.053, 'total_flos': 7081137864179712.0, 'train_loss': 4.316118636552026, 'epoch': 3.0})

In [14]:
# Save the *LoRA adapter* weights (recommended):
lora_save_path = "lora_weights"  # Directory to save the LoRA weights
model.save_pretrained(lora_save_path)

# To save the entire merged model (instruct model + LoRA - less common):
# merged_model_save_path = "path/to/save/merged_model"
# model.base_model.save_pretrained(merged_model_save_path) # Saves the merged model

## 5. Apply the learned weights to the instruct model

In [15]:
# Load the instruct model
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # Or your instruct model
instruct_tokenizer = AutoTokenizer.from_pretrained(model_name)
instruct_model = AutoModelForCausalLM.from_pretrained(model_name,
    quantization_config=quantization_config,)

# Load the LoRA configuration and weights
lora_weights_path = "lora_weights"  # Path to your saved LoRA weights
peft_config = PeftConfig.from_pretrained(lora_weights_path)
lora_instruct_model = PeftModel.from_pretrained(instruct_model, lora_weights_path)  # Apply LoRA adapter to the instruct model

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## 6. Test the Model

Let's test the fine-tuned model with some domain-specific queries.

In [16]:
def test_qanda(model, tokenizer):
    # Example queries about heat transfer and fluid dynamics
    queries = [
        "Explain the CFD/SI method.",
        "In the case of a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what physical mechanism causes the initial negative heat transfer response in the downstream sector (sector 4) after a step increase in flow velocity?",
        "For a cylinder in cross flow, above which Reynolds number is vortex shedding occuring?",
        "What is a Rijke tube?",
        "How is a finite impulse response computed?",
        "For a heated cylinder in cross-flow under laminar conditions, what are the three distinct response regimes identified in the paper based on the interaction between velocity and temperature diffusion timescales, and what Reynolds number ranges characterize each regime?",
        "How does the frequency response function G(Sr) of heat transfer from a heated cylinder in pulsating cross-flow differ from what would be expected from a simple first-order system, particularly regarding phase behavior at high frequencies? What physical mechanism explains this difference?",
        "When analyzing heat transfer from a heated cylinder in pulsating cross-flow using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?",
        "For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?",
    ]
    
    # Generate responses
    for query in queries:
        print(f"\nQuery: {query}")
        response = generate_response(
            model,
            tokenizer,
            query,
            max_new_tokens=512,
            temperature=0.7
        )
        print(f"Response: {response}")
        print("-" * 80)

In [17]:
test_qanda(instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: CF/SI is a method for heat and mass transfer in fluids in flows. is to. the transfer of energy from fluid to solid surface transfer from fluid to fluid transfer heat from fluid to fluid the C/S method a of. to the, transfer heat fluid fluid transfer fluid heat transfer heat fluid fluid transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer t

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Response: ,,.,.
## Step. Identify the regimes based on interaction between velocity and temperature diffusion timescales.
 the regimes are identified as,  and, based the interaction between velocity temperature diffusion timescales as,   and  respectively.

##. Determine Reynolds number range for each regime.
 regime : 0 < < (,., regime :  < (,., regime :  > (,.

The final answer is: 0    ................. ....... ...... ..... .  .... ....  .. .. ........ .   .
--------------------------------------------------------------------------------

Query: How does the frequency response function G(Sr) of heat transfer from a heated cylinder in pulsating cross-flow differ from what would be expected from a simple first-order system, particularly regarding phase behavior at high frequencies? What physical mechanism explains this difference?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: .
,,.,,.-.,.,.,,,.,.,.,.,.,.,,.,.,.,.,.,.,.,,.,.,.,,,,.,.,.,.,,,.,.,.,,.,.,,.,.,.,.,,.,,.,.,,,.,,,,,,,,,. and.,.,.,.,.,.,.,,.,.,,,.,,,.,,.,,,,.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--------------------------------------------------------------------------------

Query: When analyzing heat transfer from a heated cylinder in pulsating cross-flow using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: ,,

## Step 1: Identify the context of system identification in heat transfer
System identification is a technique used to identify the of a system from data., The context here is heat transfer from a heated cylinder puls cross. systems are of interest in various applications engineering,.

## Step 2 Determine key for validation
 key for validation include accuracy accuracy of data used model validation robustness of models ability handle uncertainties noise effects andability replicate original behavior andability generalize to new conditions.

##3 Identify minimum threshold
 minimum threshold considered acceptable depends on application goals and requirements but common is performance threshold % error acceptable.. for heat transfer a common is % error, which measures difference between model output actual and output.

## Step4 Threshold value
 threshold value  depends application but common is10. this means model accuracy within10 of actual is acceptable for purposes syste

In [18]:
test_qanda(lora_instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: C/SI is a computational approach for in simulations. It the to a time in the domain and then the is computed the for the time.,,. The is computed the for the at each point the.,. the is, the time, and the is.,. the is the and the is the.,. the method used CFDI to simulations the flow a cylinder a around a and a., the method usedFDI simulations the around a cylinder a and a the., the was to a in the and the is.,. the method usedFDI the a flow a around a and a the and the the in the was.,. the usedFDI the a flow a a around a and a the and the the in the was, and the the of the was and the was,., The is the to a in the and the is., the is the to a in and the is.,. theFDI used theFDI the a around a and a the and the the in the., The is the to a in and the is,, theFDI usedFDI the around a and a the and the the the., The is the to a and the is and theFDI theFDI the a and a the the., TheFDI theFDI the a and a the and the the in the,. The is the to a in the and the is andFDIFDI theFD

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Response: ,,.,. The isic of response is given byτ vτt, where v is velocity,t is time and is the fluid.0.0 003.. .... ...03.0...............     ..0 ....... .........     ..0........ .  ......    ...   ....  .  ..  . ....    ... ...      .. .       . ...    . ....   ..  . .. ... .. .  .    . ....     .  . . .    . ..  . ...   .  ..         . ...  . ..      .  .. ... . ..        ....  ...  ...  ...    . ......   ..   .   .   ..... .   . ..   ..
--------------------------------------------------------------------------------

Query: How does the frequency response function G(Sr) of heat transfer from a heated cylinder in pulsating cross-flow differ from what would be expected from a simple first-order system, particularly regarding phase behavior at high frequencies? What physical mechanism explains this difference?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: _.STEP._...... .
--------------------------------------------------------------------------------

Query: When analyzing heat transfer from a heated cylinder in pulsating cross-flow using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: ,,

## Step . The system identification process for heat transfer from a heated cylinder in puls cross-flow typically involves several steps data acquisition data processing identification and validation The key criteria used validate identified models include model accuracy convergence robustness and minimization residuals The minim threshold considered acceptable depends the application the context and desired performance of system In general a threshold accepted acceptable depends the of steady-state oscill and amplitude oscill during flow Additionally validation also performed checks model with experimental or simulations data model is to certain known models and models are compared mean squared errorMS and root mean squaredRM values models with better values considered acceptable

## Step2 The validation process system identification typically involves comparison model experimental data simulations results Model is validated if model closely matches data simulations and 

## 7. Save the Model

Finally, let's save the fine-tuned model for later use.

In [19]:
# Save the fine-tuned model
output_dir = Path("./final_model")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Model saved to final_model
